In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import random
import os
import pickle
from sklearn import metrics
from datetime import date
from sklearn.metrics import classification_report
import Evaluation_function_for_batch as func
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots

In [2]:
file_path = 'D:\\庫存健診開發\\data\\prediction\\week_whole\\'
origin_path = 'D:\\庫存健診開發\\data\\stock_processed.csv'

prediction, df = func.read_prediction(file_path, origin_path)
df

,ts,StockNo,total,Y,Y_0_score,Y_1_score
0,2017-09-01,1109,411000.0,1.0,0.462982,0.537018
1,2017-09-04,1109,1068000.0,1.0,0.641480,0.358520
2,2017-09-05,1109,1187000.0,1.0,0.483319,0.516681
3,2017-09-06,1109,469000.0,1.0,0.653811,0.346189
4,2017-09-07,1109,1727000.0,1.0,0.280490,0.719510
...,...,...,...,...,...,...
453894,2019-09-09,9955,16507000.0,0.0,0.738698,0.261303
453895,2019-09-10,9955,25898000.0,0.0,0.871708,0.128292
453896,2019-09-11,9955,31263000.0,0.0,0.910328,0.089672
453897,2019-09-12,9955,15657000.0,0.0,0.725949,0.274051


In [3]:
len(df['StockNo'].unique())

931

In [4]:
data = func.read_original_data('D:\\庫存健診開發\\data\\stock_processed.csv')

In [5]:
d = pd.merge(data, df, on=['ts', 'StockNo'], how='inner')
d['verify'] = d.apply(func.verify, axis=1)
d[~d.verify]

,ts,StockNo,close,foreign_ratio,investment_ratio,corporation_ratio,index_close,VWAP_day5,VWAP_after,close_return,future_return,total,Y,Y_0_score,Y_1_score,verify


In [16]:
df_list = [group[1] for group in df.groupby(df['StockNo'])]
output_list = []
for d in df_list:
    d['total_mean'] = d[d.ts.dt.year == 2019]['total'].mean()
    output_list.append(d)
    
df = pd.concat(output_list, axis=0)

In [20]:
pd.Series(df.total_mean.unique()).quantile(0.4)

8233366.8639053255

In [17]:
subdf = df[df.total_mean > pd.Series(df.total_mean.unique()).quantile(0.4)]

In [18]:
len(subdf['StockNo'].unique())

558

In [19]:
test_year = [2017, 2018, 2019]
threshold = [0.5, 0.55, 0.6, 0.65]

In [20]:
# q = df['Y_1_score'].quantile(np.linspace(.1, 1, 9, 0))
for t in threshold:
    r = func.Evaluation(subdf, threshold=t)
    print(f'The threshold is {t}')
    print(f'Accuracy: {r[0]} \n up_support/ratio: {r[2]}/{r[4]} \n down_support/ratio: {r[3]}/{r[5]}')
    print(r[1])

The threshold is 0.5
Accuracy: 0.7003928222341073 
 up_support/ratio: 121124/0.44676244398133635 
 down_support/ratio: 149991/0.5532375560186636
              precision    recall  f1-score   support

        down       0.68      0.76      0.71    134095
          up       0.73      0.65      0.69    137020

    accuracy                           0.70    271115
   macro avg       0.70      0.70      0.70    271115
weighted avg       0.70      0.70      0.70    271115

The threshold is 0.55
Accuracy: 0.7254887068940056 
 up_support/ratio: 104204/0.38435350312597977 
 down_support/ratio: 132799/0.4898253508658687
              precision    recall  f1-score   support

        down       0.70      0.79      0.74    118773
          up       0.76      0.67      0.71    118230

    accuracy                           0.73    237003
   macro avg       0.73      0.73      0.72    237003
weighted avg       0.73      0.73      0.72    237003

The threshold is 0.6
Accuracy: 0.7511406956762202 
 up_

In [9]:
for y in test_year:
    for t in threshold:
        r = func.Separate_Evaluation(df, prediction=True, threshold=t, year=y)
        print(f'The threshold is {t}, year {y}')
        print(f'Accuracy: {r[0]} \n up_support/ratio: {r[2]}/{r[4]} \n down_support/ratio: {r[3]}/{r[5]}')
        print(r[1])

The threshold is 0.5, year 2017
Accuracy: 0.678043844856661 
 up_support/ratio: 30995/0.4181450252951096 
 down_support/ratio: 43130/0.5818549747048903
              precision    recall  f1-score   support

        down       0.66      0.76      0.70     37555
          up       0.70      0.60      0.65     36570

    accuracy                           0.68     74125
   macro avg       0.68      0.68      0.68     74125
weighted avg       0.68      0.68      0.68     74125

The threshold is 0.55, year 2017
Accuracy: 0.7044684422206036 
 up_support/ratio: 25623/0.345672849915683 
 down_support/ratio: 37531/0.5063204047217538
              precision    recall  f1-score   support

        down       0.68      0.79      0.73     32537
          up       0.73      0.61      0.67     30617

    accuracy                           0.70     63154
   macro avg       0.71      0.70      0.70     63154
weighted avg       0.71      0.70      0.70     63154

The threshold is 0.6, year 2017
Accuracy:

In [10]:
d_stock = func.Stock_evaluation(data, df, threshold=0.5)
d_stock['accuracy'].min()

0.57

In [13]:
for col in ['accuracy', 'up_precision', 'down_precision']:
    print(f'{col} min: {d_stock[col].min()}')
    print(f'{col} max: {d_stock[col].max()}')
    print(f'{col} mean: {d_stock[col].mean()}')
    print(f'{col} std: {d_stock[col].std()}')

accuracy min: 0.57
accuracy max: 0.7988165680473372
accuracy mean: 0.6952726283650816
accuracy std: 0.028899197375027722
up_precision min: 0.45121951219512196
up_precision max: 0.8888888888888888
up_precision mean: 0.7064092432416633
up_precision std: 0.05875862642452793
down_precision min: 0.48717948717948717
down_precision max: 0.8485804416403786
down_precision mean: 0.6810986031203302
down_precision std: 0.05185431682947345


In [20]:
df_list1 = []
q = df['Y_1_score'].quantile(np.linspace(.1, 1, 9, 0))
for t in q.iloc[5:]:
    d = func.daily_evaluation(df, threshold=t)
    display(d)
    df_list1.append(d)

TypeError: daily_evaluation() missing 1 required positional argument: 'prediction'

In [14]:
d1 = func.monthly_evaluation(data, df, threshold=0.5)
d1 = d1.sort_index()
d1

,accuracy,up_return,up_ratio,up_portfolio,down_return,down_ratio,down_portfolio,up_precision,down_precision,up_support_mean,down_support_mean,index
2017-09-01,0.683636,0.784219,0.0185459,0.0136727,0.70772,0.00875083,0.0140546,0.693744,0.677875,318.955,559.636,10383.9
2017-10-01,0.681278,1.04583,0.016348,0.018635,0.391129,0.00537847,0.00768909,0.705663,0.657116,436.842,440.842,10793.8
2017-11-01,0.675935,0.823888,0.0169667,0.0152966,1.01044,0.0109793,0.0188989,0.65977,0.68705,360.318,524.045,10560.4
2017-12-01,0.67156,0.883694,0.0190489,0.0177402,0.873937,0.00625206,0.0163882,0.760547,0.608357,369.095,519.667,10642.9
2018-01-01,0.723461,0.774118,0.016324,0.0140505,0.787171,0.0147728,0.015618,0.68157,0.762893,433.091,460.091,11103.8
2018-02-01,0.746845,1.34485,0.0290012,0.0250059,1.28535,0.0228581,0.0261025,0.795066,0.710252,386.615,509.462,10815.5
2018-03-01,0.681109,1.21331,0.0207729,0.0225932,0.633213,0.00761383,0.0122143,0.713819,0.655071,398.957,501.174,10919.5
2018-04-01,0.709968,1.10235,0.0125449,0.0195737,1.06829,0.0186684,0.020882,0.594805,0.767326,299.444,601.222,10657.9
2018-05-01,0.655725,1.74544,inf,0.0308945,0.104473,0.00246944,0.00209033,0.739359,0.582247,422.909,481.364,10875
2018-06-01,0.728177,1.35893,0.017726,0.0225566,1.07334,0.0171727,0.0203557,0.655518,0.775955,358.8,545.65,10836.9


In [15]:
for col in ['accuracy', 'up_precision', 'up_support_mean', 'down_precision', 'down_support_mean']:
    print(f'{col} min: {d1[col].min()}')
    print(f'{col} max: {d1[col].max()}')
    print(f'{col} mean: {d1[col].mean()}')
    print(f'{col} std: {d1[col].std()}')

accuracy min: 0.6557253443249221
accuracy max: 0.7680181314530345
accuracy mean: 0.6967546049728875
accuracy std: 0.02725063197308294
up_precision min: 0.5368611488014473
up_precision max: 0.8353363378400934
up_precision mean: 0.703088755915101
up_precision std: 0.07596771152905836
up_support_mean min: 201.0
up_support_mean max: 610.2307692307693
up_support_mean mean: 399.3063190488637
up_support_mean std: 86.71610348862943
down_precision min: 0.4964921072412929
down_precision max: 0.834251279725264
down_precision mean: 0.6720846243215989
down_precision std: 0.084439887633666
down_support_mean min: 313.7692307692308
down_support_mean max: 701.5
down_support_mean mean: 509.01530035690206
down_support_mean std: 82.89539500969738


In [3]:
df_list1 = []
q = df['Y_1_score'].quantile(np.linspace(.1, 1, 9, 0))
for t in q.iloc[5:]:
    d = func.monthly_evaluation(df, threshold=t)
    display(d)
    df_list1.append(d)

,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support_mean,down_support_mean,index
2019-10-01,0.721686,1.95665,0.0229963,0.024047,0.818794,0.638213,205.238,238.762,11358.7


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support_mean,down_support_mean,index
2019-10-01,0.758429,2.31141,0.0264191,0.0280544,0.85184,0.681876,153.952,187.857,11358.7


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support_mean,down_support_mean,index
2019-10-01,0.796704,2.81345,0.0313103,0.0330617,0.886311,0.725185,102.619,128.571,11358.7


,accuracy,return,ratio,portfolio,up_precision,down_precision,up_support_mean,down_support_mean,index
2019-10-01,0.850265,3.94625,0.0409348,0.0409973,0.923006,0.793154,51.3333,65.381,11358.7


In [65]:
# df1 = d[d.StockNo == '2330'].sort_values(by='ts').reset_index(drop=True)
# df1['future_return']
fig2 = make_subplots(rows=2, cols=1)
# fig2 = go.Figure()
fig2.append_trace(
    go.Scatter(
        x=d1.index,
        y=(d1['down_ratio'] + d1['up_ratio']) / 2,
        mode="lines",
        line=go.scatter.Line(color="blue"),
        name="平均報酬率(多空)",
        ),
    
    row=1, col=1,
)

fig2.append_trace(
    go.Scatter(
        x=d1.index,
        y=d1['index'],
        mode="lines",
        line=go.scatter.Line(color="gray"),
        name="大盤指數收盤價",
        ),
    
    row=2, col=1,
)

# fig2.append_trace(
#     go.Scatter(
#         x=d1.index,
#         y=d1['down_ratio'],
#         mode="lines",
#         line=go.scatter.Line(color="green"),
#         name="down_ratio",
#         ),
    
#     row=2, col=1,
# )

# fig2.append_trace(
#     go.Scatter(
#         x=d1.index,
#         y=(d1['down_ratio'] + d1['up_ratio'])/2,
#         mode="lines",
#         line=go.scatter.Line(color="blue"),
#         name="total_ratio",
#         ),
    
#     row=3, col=1,
# )

# fig = go.Figure()


# fig.add_trace(
#     go.Scatter(
#         x=d1.index,
#         y=d1['index'],
#         mode="lines",
#         line=go.scatter.Line(color="gray"),
#         name="index_close",
#         ),
    
# #     row=1, col=1,
# )
# fig.update_layout(showlegend=True)




# fig.update_layout(xaxis_rangeslider_visible=False, width=800)
# fig2.update_layout(width=835,height=300)
# fig2.update_yaxes(range=[0, 0.07])

fig2.show()
# fig.show()



In [15]:
for y in test_year:
    for df4 in df_list:
        d, r = func.daily_aggregate(df4, year=y)
        print(f'The threshold is {t}, year {y}')
        display(d)
        display(r['daily_accuracy_worst'])
        display(r['daily_ratio_worst'])
        display(r['daily_portfolio_worst'])

AttributeError: 'DataFrame' object has no attribute 'ts'

In [28]:
df_index = func.get_daily_index(date(2017,9,1), date(2019,9,6))
df_index = df_index.sort_values(by='ts').reset_index(drop=True)

In [29]:
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots


df1 = func.monthly_evaluation(data, df, threshold=0.5)
df1 = df1.sort_index()
fig2 = make_subplots(rows=3, cols=1)

fig2.append_trace(
    go.Scatter(
        x=df1.index,
        y=df1['up_precision'],
        mode="lines",
        line=go.scatter.Line(color="gray"),
        name="up_precision",
        ),
    
    row=1, col=1,
)

fig2.append_trace(
    go.Scatter(
        x=df1.index,
        y=df1['down_precision'],
        mode="lines",
        line=go.scatter.Line(color="gray"),
        name="down_precision",
        ),
    
    row=2, col=1,
)

fig2.append_trace(
    go.Scatter(
        x=df_index['ts'],
        y=df_index['index_close'],
        mode="lines",
        line=go.scatter.Line(color="blue"),
        name="index",
        ),
    
    row=3, col=1,
)

# fig.update_layout(xaxis_rangeslider_visible=False, width=800)
# fig2.update_layout(width=835,height=300)
# fig2.update_yaxes(range=[0, 1])

fig2.show()
# fig.show()


In [25]:
fig = go.Figure(data=[go.Candlestick(x=df_index['ts'],
                open=df_index['index_open'],
                high=df_index['index_high'],
                low=df_index['index_low'],
                close=df_index['index_close'],
                increasing_line_color= 'red', decreasing_line_color= 'green')])
fig.show()

In [3]:
val = func.read_json('D:\\庫存健診開發\\data\\prediction\\15days\\15day_val.json')
val['ts'] = pd.to_datetime(val['ts'])
val = val[val.ts.dt.date > date(2016,10,19)]
val

,Y,ts,StockNo,cluster,Y_0_score,Y_1_score
0,0,2017-05-18,3419,0,0.500544,0.499456
1,1,2017-05-18,4551,0,0.501964,0.498036
2,0,2017-05-18,3346,0,0.580022,0.419978
3,0,2017-05-18,5288,0,0.751034,0.248966
4,1,2017-05-18,3437,0,0.692676,0.307324
...,...,...,...,...,...,...
111332,0,2016-11-04,3035,4,0.720575,0.279425
111333,0,2016-11-04,1305,4,0.378397,0.621603
111334,0,2016-11-04,1476,4,0.553667,0.446333
111335,0,2016-11-04,1338,4,0.753564,0.246436


In [4]:
val.sort_values(by='ts')

,Y,ts,StockNo,cluster,Y_0_score,Y_1_score
2875,1,2016-10-20,2496,0,0.499905,0.500095
38385,1,2016-10-20,1457,1,0.579245,0.420755
38384,0,2016-10-20,2481,1,0.733806,0.266194
38383,0,2016-10-20,2514,1,0.608397,0.391603
38382,1,2016-10-20,1455,1,0.612901,0.387099
...,...,...,...,...,...,...
10359,1,2017-08-30,2405,0,0.474569,0.525431
10360,1,2017-08-30,3013,0,0.672883,0.327117
10361,1,2017-08-30,3049,0,0.490660,0.509340
10353,1,2017-08-30,2368,0,0.647712,0.352288


In [5]:
val_year = [2016, 2017]
threshold = [0.5, 0.6]

In [6]:
for t in threshold:
    r = func.Evaluation(val, threshold=t)
    print(f'The threshold is {t}')
    print(f'Accuracy: {r[0]} \n up_support/ratio: {r[2]}/{r[4]} \n down_support/ratio: {r[3]}/{r[5]}')
    print(r[1])

The threshold is 0.5
Accuracy: 0.6259169365971614 
 up_support/ratio: 51501/0.4646804594382438 
 down_support/ratio: 59330/0.5353195405617562
              precision    recall  f1-score   support

        down       0.61      0.66      0.64     54730
          up       0.64      0.59      0.61     56101

    accuracy                           0.63    110831
   macro avg       0.63      0.63      0.63    110831
weighted avg       0.63      0.63      0.63    110831

The threshold is 0.6
Accuracy: 0.686110517754153 
 up_support/ratio: 23449/0.2115743790094829 
 down_support/ratio: 37410/0.3375409407115338
              precision    recall  f1-score   support

        down       0.67      0.79      0.72     31573
          up       0.72      0.57      0.64     29286

    accuracy                           0.69     60859
   macro avg       0.69      0.68      0.68     60859
weighted avg       0.69      0.69      0.68     60859



In [7]:
for y in val_year:
    for t in threshold:
        r = func.Separate_Evaluation(val, prediction=True, threshold=t, year=y)
        print(f'The threshold is {t}, year {y}')
        print(f'Accuracy: {r[0]} \n up_support/ratio: {r[2]}/{r[4]} \n down_support/ratio: {r[3]}/{r[5]}')
        print(r[1])

The threshold is 0.5, year 2016
Accuracy: 0.6203911870503597 
 up_support/ratio: 11077/0.4150554556354916 
 down_support/ratio: 15611/0.5849445443645084
              precision    recall  f1-score   support

        down       0.63      0.69      0.66     14060
          up       0.61      0.54      0.57     12628

    accuracy                           0.62     26688
   macro avg       0.62      0.62      0.62     26688
weighted avg       0.62      0.62      0.62     26688

The threshold is 0.6, year 2016
Accuracy: 0.6867543383947939 
 up_support/ratio: 4715/0.17667116306954436 
 down_support/ratio: 10037/0.3760866306954436
              precision    recall  f1-score   support

        down       0.68      0.83      0.75      8214
          up       0.70      0.51      0.59      6538

    accuracy                           0.69     14752
   macro avg       0.69      0.67      0.67     14752
weighted avg       0.69      0.69      0.68     14752

The threshold is 0.5, year 2017
Accuracy

In [9]:
for t in threshold:
    r, s, c, a, count = func.Overall_Evaluation(val, predict_threshold=t)
    print(f'The threshold is {t}')
    print(f'Ratio < 0.6: {r} \n cluster ratio: {count} \n variance: {np.array(a).std()}')


The threshold is 0.5
Ratio < 0.6: 0.2784090909090909 
 cluster ratio: [0.3473684210526316, 0.2826086956521739, 0.22321428571428573, 0.2727272727272727, 0.27350427350427353] 
 variance: 0.052096296527091794



The threshold is 0.6
Ratio < 0.6: 0.08333333333333333 
 cluster ratio: [0.09473684210526316, 0.09420289855072464, 0.044642857142857144, 0.09090909090909091, 0.09401709401709402] 
 variance: 0.0638892203637639


In [6]:
for t in threshold:
    d, r = func.daily_ratio(data, val, threshold=t)
    print(f'The threshold is {t}')
    display(d)
    display(r['daily_accuracy_worst'])
    display(r['daily_ratio_worst'])
    display(r['daily_portfolio_worst'])


The threshold is 0.5


,min,max,std,mean,min_date,max_date,worst,worst_date,best,best_date
daily_accuracy,0.410058,0.863014,0.0718448,0.625946,2016-12-27 00:00:00,2016-11-09 00:00:00,5,2016-12-29 00:00:00,49,2017-08-18 00:00:00
daily_up_count,56,417,76.9192,241.789,2016-11-08 00:00:00,2017-02-15 00:00:00,17,2016-11-11 00:00:00,96,2017-04-12 00:00:00
daily_down_count,103,455,76.2084,278.545,2017-02-15 00:00:00,2016-11-08 00:00:00,20,2017-01-24 00:00:00,71,2017-06-05 00:00:00
daily_return,-1.87135,3.97482,1.06207,0.963313,2016-11-14 00:00:00,2017-04-26 00:00:00,6,2016-12-27 00:00:00,39,2017-03-02 00:00:00
daily_ratio,-0.0292898,0.0581851,0.0140387,0.0183934,2017-08-14 00:00:00,2017-08-21 00:00:00,5,2016-12-28 00:00:00,55,2017-03-27 00:00:00
daily_portfolio,-0.0331252,0.0470995,0.0161646,0.0160605,2016-11-14 00:00:00,2017-07-12 00:00:00,6,2016-12-27 00:00:00,39,2017-03-02 00:00:00


[0.501937984496124,
 0.48255813953488375,
 0.41005802707930367,
 0.4816247582205029,
 0.5241779497098646]

[-0.008683076098044078,
 -0.016388517094452017,
 -0.005747330309953209,
 -0.010774955197292253,
 -0.003991971942772654]

[-0.0019891899065578615,
 -0.007349858099135135,
 -0.01390855178751051,
 -0.02598291304555531,
 -0.012454359286042918,
 -0.010320054286585834]


The threshold is 0.6


,min,max,std,mean,min_date,max_date,worst,worst_date,best,best_date
daily_accuracy,0.409091,0.920213,0.0924131,0.681915,2017-04-27 00:00:00,2016-11-09 00:00:00,5,2016-10-27 00:00:00,42,2017-08-18 00:00:00
daily_up_count,14,263,50.3056,110.089,2016-11-08 00:00:00,2017-02-13 00:00:00,10,2016-11-11 00:00:00,89,2017-03-30 00:00:00
daily_down_count,48,375,74.6074,175.634,2017-02-15 00:00:00,2016-11-03 00:00:00,19,2017-01-20 00:00:00,68,2017-06-06 00:00:00
daily_return,-1.0755,3.73806,0.825219,1.29671,2017-08-14 00:00:00,2017-04-28 00:00:00,4,2016-12-27 00:00:00,126,2017-07-07 00:00:00
daily_ratio,-0.0193065,0.0796779,0.0156094,0.0291428,2017-08-14 00:00:00,2017-08-21 00:00:00,2,2016-12-23 00:00:00,150,2017-08-11 00:00:00
daily_portfolio,-0.0266247,0.0632873,0.0149388,0.0272319,2017-08-14 00:00:00,2017-04-21 00:00:00,4,2016-12-27 00:00:00,126,2017-07-07 00:00:00


[0.41975308641975306,
 0.5017301038062284,
 0.528169014084507,
 0.5656934306569343,
 0.5808383233532934]

[-0.0004930378245669043, -0.00824001457465935]

[-0.00249962052773574,
 -0.01199408371207399,
 -0.0016878091259105393,
 -0.003780233066067121]

In [9]:
for y in val_year:
    for t in threshold:
        d,r = func.daily_ratio(data, val, threshold=t, year=y)
        print(f'The threshold is {t}, year {y}')
        display(d)
        display(r['daily_accuracy_worst'])
        display(r['daily_ratio_worst'])
        display(r['daily_portfolio_worst'])


The threshold is 0.5, year 2016


,min,max,std,mean,min_date,max_date,worst,worst_date,best,best_date
daily_accuracy,0.410058,0.863014,0.105302,0.620573,2016-12-27 00:00:00,2016-11-09 00:00:00,4,2016-12-28 00:00:00,20,2016-11-21 00:00:00
daily_up_count,56,351,83.9768,213.019,2016-11-08 00:00:00,2016-12-30 00:00:00,10,2016-11-10 00:00:00,5,2016-10-27 00:00:00
daily_down_count,167,455,81.5535,300.212,2016-12-30 00:00:00,2016-11-08 00:00:00,0,2016-10-20 00:00:00,48,2016-12-26 00:00:00
daily_return,-1.87135,1.69665,0.783672,0.453225,2016-11-14 00:00:00,2016-12-01 00:00:00,6,2016-12-27 00:00:00,22,2016-12-19 00:00:00
daily_ratio,-0.0175743,0.0499723,0.0153321,0.0159858,2016-11-03 00:00:00,2016-11-16 00:00:00,5,2016-12-28 00:00:00,30,2016-12-21 00:00:00
daily_portfolio,-0.0331252,0.0441094,0.0175748,0.0116118,2016-11-14 00:00:00,2016-11-16 00:00:00,6,2016-12-27 00:00:00,22,2016-12-19 00:00:00


[0.501937984496124,
 0.48255813953488375,
 0.41005802707930367,
 0.4816247582205029]

[-0.008683076098044078,
 -0.016388517094452017,
 -0.005747330309953209,
 -0.010774955197292253,
 -0.003991971942772654]

[-0.0019891899065578615,
 -0.007349858099135135,
 -0.01390855178751051,
 -0.02598291304555531,
 -0.012454359286042918,
 -0.010320054286585834]

...


The threshold is 0.6, year 2017


,min,max,std,mean,min_date,max_date,worst,worst_date,best,best_date
daily_accuracy,0.409091,0.886154,0.0825942,0.684162,2017-04-27 00:00:00,2017-04-17 00:00:00,4,2017-05-02 00:00:00,33,2017-03-17 00:00:00
daily_up_count,30,263,51.3642,116.36,2017-04-13 00:00:00,2017-02-13 00:00:00,13,2017-04-28 00:00:00,63,2017-08-21 00:00:00
daily_down_count,48,327,69.0275,170.019,2017-02-15 00:00:00,2017-05-11 00:00:00,14,2017-01-20 00:00:00,68,2017-06-06 00:00:00
daily_return,-1.0755,3.73806,0.823786,1.461,2017-08-14 00:00:00,2017-04-28 00:00:00,1,2017-07-10 00:00:00,123,2017-07-07 00:00:00
daily_ratio,-0.0193065,0.0796779,0.0148187,0.029939,2017-08-14 00:00:00,2017-08-21 00:00:00,2,2017-08-15 00:00:00,148,2017-08-11 00:00:00
daily_portfolio,-0.0266247,0.0632873,0.0140397,0.0288212,2017-08-14 00:00:00,2017-04-21 00:00:00,1,2017-07-10 00:00:00,123,2017-07-07 00:00:00


[0.4483985765124555,
 0.4090909090909091,
 0.5201465201465202,
 0.5119453924914675]

[-0.019306549557967494, -0.00018005650757624983]

[-0.0025901545248284905]

In [10]:
for t in threshold:
    d = func.winpoint(data, val, threshold=t)
    print(f'The threshold is {t}')
    display(d)

D:\庫存健診開發\code\PredictModel\Evaluation_function.py:231: RuntimeWarning: invalid value encountered in double_scalars
  df.loc[index, 'portfolio'] = d['return'].sum() / d['close_lag'].sum()


The threshold is 0.5


,return_mean,return_std,ratio_mean,ratio_std,portfolio,mean,std,ratio
up,NaN,NaN,NaN,0.0470606,0.017015,0.982178,8.923790,0.018716
down,NaN,NaN,NaN,0.035016,-0.00452458,-0.262358,7.251431,-0.006482
neutral,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The threshold is 0.6


,return_mean,return_std,ratio_mean,ratio_std,portfolio,mean,std,ratio
up,NaN,NaN,NaN,0.0534907,0.0273662,1.246073,4.416817,0.028315
down,NaN,NaN,NaN,0.0344009,-0.0115533,-0.597986,3.590127,-0.011261
neutral,NaN,NaN,NaN,0.0374704,0.00826282,0.563694,11.200899,0.006737


In [6]:
for y in val_year:
    for t in threshold:
        d = func.winpoint(data, df, threshold=t, year=y)
        print(f'The threshold is {t}, year {y}')
        display(d)

The threshold is 0.5, year 2017


,mean,std,ratio,portfolio
up,0.986095,8.97327,0.0225988,0.0150229
down,-0.876554,15.5051,-0.00835533,-0.0134675
neutral,NaN,NaN,NaN,NaN


The threshold is 0.6, year 2017


,mean,std,ratio,portfolio
up,1.6198,5.93243,0.0339923,0.0284062
down,-0.973536,12.4569,-0.0114951,-0.0170771
neutral,-0.195744,17.3646,0.00444916,-0.00238939


The threshold is 0.5, year 2018


...

,mean,std,ratio,portfolio
up,1.22761,11.4376,0.0290053,0.0218207
down,-1.24386,7.86431,-0.0222944,-0.0230525
neutral,0.164313,16.9707,0.00405297,0.00202573


The threshold is 0.5, year 2019


,mean,std,ratio,portfolio
up,1.28657,9.68,0.0206834,0.0199768
down,-0.550351,9.97228,-0.0105077,-0.00844594
neutral,NaN,NaN,NaN,NaN


The threshold is 0.6, year 2019


,mean,std,ratio,portfolio
up,1.55836,8.63045,0.0268298,0.0265391
down,-0.771892,6.76411,-0.0149432,-0.0132489
neutral,0.389206,13.0915,0.00449875,0.00499895
